In [1]:
import xml.etree.ElementTree as ET
from PIL import Image
import os
import pandas as pd
import json
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split, DataLoader, Dataset

import torch
from torchvision import datasets, transforms

In [ ]:
%pip install -q transformers

Найдем все папки в которых лежит вайл с аннотацией

In [2]:

crop_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\crops"
absolute_path = "C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\data\\BallerTV"

if not os.path.exists(crop_path):
    os.makedirs(crop_path)

file_name = 'annotations.xml'

anno_pathses = []
# пройдемся рекурсивно по всем папкам, начиная с absolute_path
for root, dirs, files in os.walk(absolute_path):    
    # проверяем, есть ли заданный файл в списке файлов текущей папки
    if file_name in files:
        # если есть, то добавляем путь в список
        anno_pathses.append(os.path.join(root, file_name))      
     

In [3]:
nums = []
num = 0
pathes_list = []
annos_list = []
anno_dict = {'file_name': [], 'text': []}

for path in tqdm(anno_pathses):

    # Загрузим XML файл
    tree = ET.parse(path)

    # Получим корневой элемент XML файла
    root = tree.getroot()

    # Проитерируемся по дочерним элементам корня
    for image in root.findall('image'):
        
        image_id = image.get('id')
        file_name = image.get('name')
        file_name = 'data\\' + file_name

        # Переберем все baundingbox`s
        for box in image.findall('box'):
            
            image = Image.open(file_name)
           
            label = box.get('label')

            # Проверим является ли метка меткой номера
            if label == "Jersey":
                xtl = int(float(box.get('xtl')))
                ytl = int(float(box.get('ytl')))
                xbr = int(float(box.get('xbr')))
                ybr = int(float(box.get('ybr')))
                number = box.find('attribute[@name="number"]').text

                # Сделаем кроп номера на футболке
                croppped_image = image.crop((xtl, ytl, xbr, ybr))
                full_path = crop_path + '\\' + str(number)
                num += 1

                if not os.path.exists(full_path):
                    os.makedirs(full_path)    
                    #print(f'Папка {number} создана')  
                                  
                res_path = full_path +'\\' + str(num) + 'example.jpg'

                annos_list.append(int(number))
                pathes_list.append(res_path )
                anno_dict['text'] = annos_list
                anno_dict['file_name'] = pathes_list
                
                croppped_image.save(res_path)

                df = pd.DataFrame(anno_dict, index=None)
                df.to_csv("anno.csv")
df.head()


 21%|██        | 22/105 [01:14<04:42,  3.41s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'data\\BallerTV/basketball/3_processed/09-08-2022/frames/7683_94590_1658679588_raw/000001.jpeg'

In [4]:
df.head()

,file_name,text
0,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,1
1,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,8
2,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,11
3,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3
4,C:\Users\Mytre\OneDrive\Документы\Data\Work\cr...,3


In [5]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [6]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
        #print(type(text))
        # prepare image (i.e. resize + normalize)
        image = Image.open(file_name).convert("RGB")
        image = image.resize((32, 100))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [7]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

#processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
#model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

In [9]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 3679
Number of validation examples: 920


In [10]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### В этом блоке мы смотрим в ручном режиме как решает нашу задачу модель из коробки

In [10]:
url = "crops\\75\\3272example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Номер на футболке: 5


In [22]:
transform = transforms.Compose(
        [transforms.RandomResizedCrop(224), 
         transforms.ToTensor()])

In [11]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

In [12]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=10,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="./",
    logging_steps=50,
    save_steps=1000,
    eval_steps=200,
)

In [13]:
from datasets import load_metric

cer_metric = load_metric("cer")

C:\Users\Mytre\AppData\Local\Temp\ipykernel_7736\152175726.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  cer_metric = load_metric("cer")


In [14]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=0.01, no_deprecation_warning=True)


In [16]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12270 [00:00<?, ?it/s]

{'loss': 2.8189, 'learning_rate': 4.9808475957620214e-05, 'epoch': 0.04}
{'loss': 1.4562, 'learning_rate': 4.960472697636512e-05, 'epoch': 0.08}
{'loss': 1.2054, 'learning_rate': 4.940097799511003e-05, 'epoch': 0.12}
{'loss': 1.1171, 'learning_rate': 4.9197229013854935e-05, 'epoch': 0.16}


c:\Users\Mytre\OneDrive\Документы\Data\Work\workenv\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


  0%|          | 0/307 [00:00<?, ?it/s]

...

  0%|          | 0/307 [00:00<?, ?it/s]

{'eval_loss': 3.7863364219665527, 'eval_cer': 0.04484933426769446, 'eval_runtime': 177.1418, 'eval_samples_per_second': 5.194, 'eval_steps_per_second': 1.733, 'epoch': 9.78}
{'loss': 0.0001, 'learning_rate': 9.331703341483293e-07, 'epoch': 9.82}
{'loss': 0.0257, 'learning_rate': 7.334963325183375e-07, 'epoch': 9.86}
{'loss': 0.0362, 'learning_rate': 5.297473512632438e-07, 'epoch': 9.9}
{'loss': 0.009, 'learning_rate': 3.2599837000814994e-07, 'epoch': 9.94}


  0%|          | 0/307 [00:00<?, ?it/s]

{'eval_loss': 3.7123093605041504, 'eval_cer': 0.04555010511562719, 'eval_runtime': 176.81, 'eval_samples_per_second': 5.203, 'eval_steps_per_second': 1.736, 'epoch': 9.94}
{'loss': 0.0073, 'learning_rate': 1.2224938875305623e-07, 'epoch': 9.98}
{'train_runtime': 15861.6971, 'train_samples_per_second': 2.319, 'train_steps_per_second': 0.774, 'train_loss': 0.2888945367067241, 'epoch': 10.0}


TrainOutput(global_step=12270, training_loss=0.2888945367067241, metrics={'train_runtime': 15861.6971, 'train_samples_per_second': 2.319, 'train_steps_per_second': 0.774, 'train_loss': 0.2888945367067241, 'epoch': 10.0})

In [20]:
url = "crops\\77\\3839example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())


generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Номер на футболке: 77
